In [2]:
import numpy as np
import pandas as pd
import math
import datetime
import re

### train_auth_info file

In [32]:
person_info = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/train_auth_info.csv")

In [33]:
person_info.head()

,id_card,auth_time,phone,id
0,4****************0,2017-06-10,132****2008,501951980776722440
1,NaN,NaN,187*****500,525890212484616200
2,5****************9,2015-10-10,135****3522,599309364691472392
3,NaN,NaN,136*****651,582559913071350024
4,3****************0,2016-12-19,137****4241,599017959926534408


In [34]:
person_info.describe()

,id_card,auth_time,phone,id
count,82908,79244,117589,120929
unique,2663,1368,85851,120929
top,3****************7,2016-09-11,135*****000,496987039762223368
freq,2365,435,12,1


In [5]:
nan_num = person_info.isna().sum()
nan_num

id_card      38021
auth_time    41685
phone         3340
id               0
dtype: int64

In [69]:
#assign phone numbers to difffernet carriers 
person_info['phone'] = person_info['phone'].map(lambda x: str(x)[:3] if pd.notnull(x) else x)
DianXin = ['133','153','180','181','189','177', '173', '199']
LianTong = ['130','131','132','155','156','186','185','176', '175', ]
YiDong = ['134','135','136','137','138','139','150','151','152','157','158','159','182','183','184','188','187','147','178']
Other = ['170', '171', '168', '199']
def sim(x):
    if x in DianXin:
        return 'China Telecom'
    elif x in LianTong:
        return 'China Unicom'
    elif x in YiDong:
        return 'China Mobile'
    elif x in Other:
        return 'Other'
    else:
        return x
person_info['phone'][person_info['phone'].notnull()] = person_info['phone'][person_info['phone'].notnull()].map(lambda x: sim(x))

In [70]:
#assign users to five regions in China based on the first numebr in ID_card
# 0: id_card missing, 1: not missing
#person_info['id_card'] = person_info['id_card'].map(lambda x: str(x)[:1] if pd.notnull(x) else x)
person_info['spt_id_card'] = np.where(person_info['id_card'].isna(), 0, 1)

#0: phone number missing, 1: not missing
person_info['phone_exist'] = np.where(person_info['phone'].isna(), 0, 1)

#extract year, month, day. If it is empty, assign -1
person_info['auth_time_year'] = person_info['auth_time'].map(lambda x: str(x)[:4] if pd.notnull(x) else -1)
person_info['auth_time_month'] = person_info['auth_time'].map(lambda x: str(x)[5:7] if pd.notnull(x) else -1)
person_info['auth_time_day'] = person_info['auth_time'].map(lambda x: str(x)[8:] if pd.notnull(x) else -1)

In [8]:
# drop auth_time column
person_info = person_info.drop(columns = "auth_time")

In [71]:
person_info["phone"].unique()

array(['China Unicom', 'China Mobile', nan, 'China Telecom', 'Other'],
      dtype=object)

In [72]:
person_info.head()

,id_card,auth_time,phone,id,spt_id_card,phone_exist,auth_time_year,auth_time_month,auth_time_day
0,4****************0,2017-06-10,China Unicom,501951980776722440,1,1,2017,06,10
1,NaN,NaN,China Mobile,525890212484616200,0,1,-1,-1,-1
2,5****************9,2015-10-10,China Mobile,599309364691472392,1,1,2015,10,10
3,NaN,NaN,China Mobile,582559913071350024,0,1,-1,-1,-1
4,3****************0,2016-12-19,China Mobile,599017959926534408,1,1,2016,12,19


In [73]:
# export processed csv file
person_info.to_csv('C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/auth_info.csv', index=False)

### Processed Data

In [81]:
auth_info = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/auth_info.csv")

In [82]:
auth_info.head()

,id_card,auth_time,phone,id,spt_id_card,phone_exist,auth_time_year,auth_time_month,auth_time_day
0,4****************0,2017-06-10,China Unicom,501951980776722440,1,1,2017,6,10
1,NaN,NaN,China Mobile,525890212484616200,0,1,-1,-1,-1
2,5****************9,2015-10-10,China Mobile,599309364691472392,1,1,2015,10,10
3,NaN,NaN,China Mobile,582559913071350024,0,1,-1,-1,-1
4,3****************0,2016-12-19,China Mobile,599017959926534408,1,1,2016,12,19


In [84]:
auth_info.shape

(120929, 9)

### train_bankcard_info

In [116]:
bankcard_info = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/train_bankcard_info.csv")
bankcard_info = bankcard_info.drop(columns = "Unnamed: 0")

In [12]:
bankcard_info.head()

,bank_name,tail_num,card_type,phone,id
0,citic,9753,debit,177****8427,5.854010e+17
1,abc,2174,debit,150****4551,5.876950e+17
2,bcom,4832,credit,185****1989,6.020980e+17
3,bob,5697,debit,186****1634,5.672050e+17
4,citic,7703,debit,158****7655,5.775380e+17


In [13]:
# check duplicate values in id column
bankcard_info["id"].duplicated().any()

True

In [117]:
# get different card types each user has 
bankcard_card_count = bankcard_info.groupby(by=['id'], as_index=False)['card_type'].agg({
    'card_type_count': lambda x: len(set(x))
    })
bankcard_card_count.head()

,id,card_type_count
0,4.402520e+17,2
1,4.402540e+17,2
2,4.402560e+17,2
3,4.402600e+17,1
4,4.402630e+17,1


In [92]:
bankcard_card_count.dtypes

id                 float64
card_type_count      int64
dtype: object

In [118]:
# get the number of banks in which user has back accounts
bankcard_bank_count = bankcard_info.groupby(by=['id'], as_index=False)['bank_name'].agg({
    'bankcard_count': lambda x: len(x)
})
bankcard_bank_count.head()

,id,bankcard_count
0,4.402520e+17,7
1,4.402540e+17,8
2,4.402560e+17,10
3,4.402600e+17,4
4,4.402630e+17,1


In [21]:
# get the number of phones for each user
bankcard_phone_count = bankcard_info.groupby(by=['id'], as_index=False)['phone'].agg({
    'phone_count': lambda x: len(set(x))
})
bankcard_phone_count.head()

,id,phone_count
0,4.402520e+17,1
1,4.402540e+17,2
2,4.402560e+17,1
3,4.402600e+17,1
4,4.402630e+17,1


In [112]:
# merge datafile
bankcard_info = pd.merge(bankcard_info, bankcard_card_count, on=['id'], how='left')
bankcard_info = pd.merge(bankcard_info, bankcard_bank_count, on=['id'], how='left')
bankcard_info = pd.merge(bankcard_info, bankcard_phone_count, on=['id'], how='left')
#bankcard_info = bankcard_info.drop(columns='card_type_count_y')
bankcard_info.head()

,bank_name,tail_num,card_type,phone,id,card_type_count,bankcard_count,phone_count
0,citic,9753,debit,China Telecom,5.854010e+17,2,3,2
1,abc,2174,debit,China Mobile,5.876950e+17,2,4,1
2,bcom,4832,credit,China Unicom,6.020980e+17,2,16,2
3,bob,5697,debit,China Unicom,5.672050e+17,2,3,1
4,citic,7703,debit,China Mobile,5.775380e+17,2,6,1


In [17]:
#assign phone numbers to difffernet carriers 
bankcard_info.phone = bankcard_info.phone.map(lambda x: str(x)[:3] if pd.notnull(x) else x)
bankcard_info.phone = bankcard_info.phone.map(lambda x: sim(x) if pd.notnull(x) else x)

In [18]:
bankcard_info.head()

,bank_name,tail_num,card_type,phone,id
0,citic,9753,debit,China Telecom,5.854010e+17
1,abc,2174,debit,China Mobile,5.876950e+17
2,bcom,4832,credit,China Unicom,6.020980e+17
3,bob,5697,debit,China Unicom,5.672050e+17
4,citic,7703,debit,China Mobile,5.775380e+17


In [113]:
bankcard_info.to_csv('C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/bankcard_info.csv')

### train_credit_info

In [22]:
credit_info = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/train_credit_info.csv")

In [23]:
credit_info.head()

,credit_score,overdraft,quota,id
0,549.0,0.0,0.0,563888070781309192
1,400.0,0.0,0.0,591567740590887176
2,490.0,4955.9,5083.0,464888846169936136
3,387.0,0.0,0.0,514996919311798280
4,394.0,0.0,0.0,20160629090004442243


In [24]:
credit_info.describe()

,credit_score,overdraft,quota
count,119269.000000,119269.000000,119269.000000
mean,419.995850,1210.913202,1731.818140
std,105.144555,2658.581602,3224.829157
min,0.000000,0.000000,0.000000
25%,385.000000,0.000000,0.000000
50%,403.000000,0.000000,0.000000
75%,496.000000,730.400000,2860.000000
max,605.000000,63290.890000,53137.000000


In [25]:
credit_info.describe(include=['O'])

,id
count,120929
unique,120929
top,536262750158983432
freq,1


In [26]:
# create new features - remaining 
credit_info['remaining'] = credit_info['quota'] - credit_info['overdraft']

In [27]:
credit_info['using_rate'] = credit_info['overdraft']/(credit_info['quota'] + 1e-6)

In [28]:
# inverse credit scores
credit_info['inverse_credit_score'] = credit_info['credit_score'].map(lambda x: 605-x)

In [29]:
credit_info.head()

,credit_score,overdraft,quota,id,remaining,using_rate,inverse_credit_score
0,549.0,0.0,0.0,563888070781309192,0.0,0.000000,56.0
1,400.0,0.0,0.0,591567740590887176,0.0,0.000000,205.0
2,490.0,4955.9,5083.0,464888846169936136,127.1,0.974995,115.0
3,387.0,0.0,0.0,514996919311798280,0.0,0.000000,218.0
4,394.0,0.0,0.0,20160629090004442243,0.0,0.000000,211.0


In [114]:
credit_info.to_csv('C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/credit_info.csv')

### train_order_info

In [6]:
train_order_info = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/train_order_info.csv")
train_order_info = train_order_info.drop(columns = "Unnamed: 0")

In [7]:
train_order_info.head(4)

,amt_order,type_pay,time_order,sts_order,phone,unit_price,id,no_order_md5,name_rec_md5,product_id_md5
0,39.0,Online Pay,2017-04-15 09:21:18,Waiting for Receipt,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN
1,39.0,Online Pay,2017-04-15 09:21:18,Finished,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN
2,288.0,Online Pay,2016-10-24 08:57:06,unknown,137****8223,NaN,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,NaN
3,428.0,Online Pay,2016-10-27 20:04:48,Finished,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN


In [10]:
train_order_info.shape

(2415645, 10)

In [32]:
train_order_info.describe(include=['O'])

,type_pay,time_order,sts_order,phone,id,no_order_md5,name_rec_md5,product_id_md5
count,2415645,2359366,2415645,2292864,2415645,2359366,2203588,744759
unique,37,1699913,45,126427,120929,1934846,118077,278050
top,Online Pay,0,Finished,152****6998,606216780532158728,f0fbedcc6a2ca42e98f6ae574fa1826e,fe94ed42923d9a010afd775f4fd1b472,313ca164c92261f02a0333b06d988324
freq,1962987,166,1409865,8703,8719,64,24679,4288


In [33]:
train_order_info.describe()

,amt_order,unit_price
count,2.347456e+06,7.447850e+05
mean,6.545219e+02,2.867161e+02
std,1.481714e+05,1.298934e+04
min,-2.980000e+02,0.000000e+00
25%,4.500000e+01,1.580000e+01
50%,9.900000e+01,4.950000e+01
75%,2.150000e+02,1.090000e+02
max,1.988889e+08,5.460000e+06


In [11]:
# convert the time order feature to be standard datetime type
train_order_info['time_order'] = train_order_info['time_order'].map(lambda x: -1 if (str(x) == 'nan') else(
                                  datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
                                  if ':' in str(x) else(
                                  datetime.datetime.utcfromtimestamp(int(x[0:10]))
                                  + datetime.timedelta(hours=8))))

In [12]:
# get the latest order history
train_order_info_time_max =  train_order_info.groupby(by=['id'], as_index=False)['time_order'].agg({
    'order_time_max': lambda x: max(x)
}) 
train_order_info_time_max.head()

,id,order_time_max
0,20160328090001743085,2016-03-20 21:37:49
1,20160328160001756263,-1
2,20160328170001757443,-1
3,20160328190001761939,2016-02-21 00:40:25
4,20160329140001784605,2016-03-12 12:18:57


In [13]:
train_order_info_time_max.shape

(120929, 2)

In [14]:
# get the oldest order history
train_order_time_min = train_order_info.groupby(by=['id'], as_index=False)['time_order'].agg({
    'order_time_min': lambda x: min(x)
})
train_order_time_min.head()

,id,order_time_min
0,20160328090001743085,2015-09-11 23:00:07
1,20160328160001756263,-1
2,20160328170001757443,-1
3,20160328190001761939,2012-04-26 14:50:00
4,20160329140001784605,2013-11-27 22:50:11


In [15]:
train_order_time_min.shape

(120929, 2)

In [44]:
train_order_info['type_pay'].unique()

array(['Online Pay', 'Cash on Delivery', 'unknown', 'Mixed Pay',
       'Pick it up', 'JD Pay', 'Installment', 'Online+JD Bean',
       'Pay at Front Desk', 'Balance', 'Online+limited Dong Coupon',
       'JD Bean', 'JD Bean Mixed Payment', 'Post Office Remittance',
       'Dong Coupon Mixed Payment', 'Company Transfer',
       'Online+Dong Coupon', 'Online+Directed Dong Coupon',
       'Directed Jing Coupon', 'Prepay Online',
       'JD Bean+Dong Coupon Mixed Pay', 'Directed Dong Coupon',
       'Online+General Dong Coupon', 'Jing Coupon Mixed Pay',
       'Online+Balance', 'Balance+Limited Dong Coupon',
       'Online+Limited Jing Coupon', 'Online+General Jing Coupon',
       'Online+Jing Coupon Pay', 'Points Payment', 'General Jing Coupon',
       'All Jing Coupon', 'Limited Jing Coupon',
       'Online+Balance+Limited Dong Coupon', 'University Self Pay',
       'University Agent Pay', 'Installment(cmb)'], dtype=object)

In [16]:
# get the number of online pay for each id
train_order_online_pay = train_order_info.groupby(by=['id']).apply(
    lambda x: x['type_pay'][(x['type_pay'] == 'Online Pay').values].count()).reset_index(name='Online Pay')
train_order_online_pay.head()

,id,Online Pay
0,20160328090001743085,6
1,20160328160001756263,0
2,20160328170001757443,0
3,20160328190001761939,21
4,20160329140001784605,32


In [17]:
train_order_online_pay.shape

(120929, 2)

In [18]:
# get the number of cash on delivery for each id
train_order_cash_on_deliv = train_order_info.groupby(by=['id']).apply(
    lambda x: x['type_pay'][(x['type_pay']=='Cash on Delivery').values].count()).reset_index(name='Cash on Delivery')
train_order_cash_on_deliv.head()

,id,Cash on Delivery
0,20160328090001743085,0
1,20160328160001756263,0
2,20160328170001757443,0
3,20160328190001761939,20
4,20160329140001784605,1


In [19]:
train_order_cash_on_deliv.shape

(120929, 2)

In [23]:
#calculate sum, mean, std, skew(0 if normal db) of amt_order, I've already considered kurt, but I counldn't understand the meaning so I remove it.
order_sum = train_order_info.groupby('id')['amt_order'].sum().reset_index(name='order_sum')
order_mean = train_order_info.groupby('id')['amt_order'].mean().reset_index(name='order_mean')
order_std = train_order_info.groupby('id')['amt_order'].std().reset_index(name='order_std')
order_skew = train_order_info.groupby('id')['amt_order'].skew().reset_index(name='order_skew')
#train_order_info['order_sum'] = train_order_info.id.map(order_sum)
#train_order_info['order_mean'] = train_order_info.id.map(order_mean)
#train_order_info['order_std'] = train_order_info.id.map(order_std)
#train_order_info['order_skew'] = train_order_info.id.map(order_skew)

In [24]:
order_sum.head()

,id,order_sum
0,20160328090001743085,28183.0
1,20160328160001756263,0.0
2,20160328170001757443,0.0
3,20160328190001761939,20711.5
4,20160329140001784605,36785.9


In [93]:
train_order_info.dtypes

amt_order         float64
type_pay           object
time_order         object
sts_order          object
phone              object
unit_price        float64
id                 object
no_order_md5       object
name_rec_md5       object
product_id_md5     object
order_sum         float64
order_mean        float64
order_std         float64
order_skew        float64
pho_num             int64
rec_num             int64
dtype: object

In [9]:
#number of phones within 1 id
pho_num = train_order_info.groupby('id')['phone'].nunique()
train_order_info['pho_num'] = train_order_info.id.map(pho_num)

In [10]:
#create new feature --- number of orders each id bought for other people
rec_num = train_order_info.groupby('id')['name_rec_md5'].nunique()
train_order_info['rec_num'] = train_order_info.id.map(rec_num)
train_order_info.head()

,amt_order,type_pay,time_order,sts_order,phone,unit_price,id,no_order_md5,name_rec_md5,product_id_md5,order_sum,order_mean,order_std,order_skew,pho_num,rec_num
0,39.0,Online Pay,2017-04-15 09:21:18,Waiting for Receipt,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN,1828.0,228.5,158.817955,-0.011212,2,2
1,39.0,Online Pay,2017-04-15 09:21:18,Finished,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN,1828.0,228.5,158.817955,-0.011212,2,2
2,288.0,Online Pay,2016-10-24 08:57:06,unknown,137****8223,NaN,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,NaN,1828.0,228.5,158.817955,-0.011212,2,2
3,428.0,Online Pay,2016-10-27 20:04:48,Finished,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN,1828.0,228.5,158.817955,-0.011212,2,2
4,428.0,Online Pay,2016-10-27 20:04:48,unknown,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN,1828.0,228.5,158.817955,-0.011212,2,2


In [56]:
train_order_info['rec_num'].unique()

array([  2,   0,   1,   3,   4,   5,  12,   7,   8,   6,   9,  10,  14,
        31,  11,  15, 148,  24,  17,  13,  21,  34,  19,  16,  18,  22,
        30,  25,  33,  37,  20,  47,  40,  23,  57,  88,  41,  28,  51,
        45,  27,  44,  29,  35,  52, 121,  26,  32,  65,  68,  43,  53,
        58,  71,  54,  87], dtype=int64)

In [11]:
# merge data file
train_order_info = pd.merge(train_order_info, train_order_cash_on_deliv, on=['id'], how='left')
train_order_info = pd.merge(train_order_info, train_order_online_pay, on=['id'], how='left')
train_order_info = pd.merge(train_order_info, train_order_time_min, on=['id'], how='left')
train_order_info = pd.merge(train_order_info, train_order_info_time_max, on=['id'], how='left')
#bankcard_info = bankcard_info.drop(columns='card_type_count_y')
train_order_info.head()

,amt_order,type_pay,time_order,sts_order,phone,unit_price,id,no_order_md5,name_rec_md5,product_id_md5,order_sum,order_mean,order_std,order_skew,pho_num,rec_num,Cash on Delivery,Online Pay,order_time_min,order_time_max
0,39.0,Online Pay,2017-04-15 09:21:18,Waiting for Receipt,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
1,39.0,Online Pay,2017-04-15 09:21:18,Finished,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
2,288.0,Online Pay,2016-10-24 08:57:06,unknown,137****8223,NaN,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,NaN,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
3,428.0,Online Pay,2016-10-27 20:04:48,Finished,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
4,428.0,Online Pay,2016-10-27 20:04:48,unknown,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18


In [13]:
train_order_info.to_csv('C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/order_info.csv')

In [5]:
order_info.head()

,Unnamed: 0,amt_order,type_pay,time_order,sts_order,phone,unit_price,id,no_order_md5,name_rec_md5,...,order_sum,order_mean,order_std,order_skew,pho_num,rec_num,Cash on Delivery,Online Pay,order_time_min,order_time_max
0,0,39.0,Online Pay,2017-04-15 09:21:18,Waiting for Receipt,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,...,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
1,1,39.0,Online Pay,2017-04-15 09:21:18,Finished,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,...,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
2,2,288.0,Online Pay,2016-10-24 08:57:06,unknown,137****8223,NaN,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,...,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
3,3,428.0,Online Pay,2016-10-27 20:04:48,Finished,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,...,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
4,4,428.0,Online Pay,2016-10-27 20:04:48,unknown,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,...,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18


### train_receive_addr

In [57]:
train_receive_info = pd.read_csv('C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/train_recieve_addr_info.csv')

In [58]:
train_receive_info.head()

,addr_id,region,phone,fix_phone,id,receiver_md5
0,NaN,unknown,NaN,NaN,575150029610684424,NaN
1,137816020.0,LN,186****4600,186****4600,563888070781309192,798371eaefb014d304e8c90ba0af1d09
2,137630418.0,LN,159****1000,159****1000,563888070781309192,41c14ef94733b4587f11ffb64e58a72d
3,137816021.0,LN,186****4600,NaN,563888070781309192,798371eaefb014d304e8c90ba0af1d09
4,138135925.0,LN,189****9687,189****9687,563888070781309192,2d3e7dc595265a37f1e9eee82a15b047


In [51]:
train_receive_info.dtypes

addr_id         float64
region           object
phone            object
fix_phone        object
id               object
receiver_md5     object
dtype: object

In [106]:
# train_recieve_info['addr_id']=train_recieve_info['addr_id'].astype('object')

In [59]:
train_recieve_info.describe()

,Unnamed: 0,addr_id,id
count,261299.000000,1.686620e+05,2.612990e+05
mean,130649.000000,1.339438e+08,3.605554e+18
std,75430.668332,1.853038e+07,7.086785e+18
min,0.000000,5.764700e+04,4.402520e+17
25%,65324.500000,1.379302e+08,5.748170e+17
50%,130649.000000,1.381809e+08,5.860990e+17
75%,195973.500000,1.383917e+08,6.085460e+17
max,261298.000000,1.410507e+08,2.016080e+19


In [59]:
# One hot-encoding region
tmp_receive = pd.crosstab(train_receive_info.id, train_receive_info.region)
tmp_receive.head()

region,AH,BJ,CQ,FJ,GD,GS,GX,GZ,HA,HB,...,SH,SN,SX,TJ,TW,XJ,XZ,YN,ZJ,unknown
id,,,,,,,,,,,,,,,,,,,,,
20160328090001743085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20160328160001756263,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20160328170001757443,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20160328190001761939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20160329140001784605,0,0,6,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
#count number of receving address each id has
addr_num = train_receive_info.groupby('id')['region'].nunique()
train_receive_info['addr_num'] = train_receive_info.id.map(addr_num)

In [61]:
# count number of unique phones each ID has
fxpho_num = train_receive_info.groupby(by='id')['fix_phone'].nunique().reset_index(name='unique_fix_phone')
fxpho_num

,id,unique_fix_phone
0,20160328090001743085,0
1,20160328160001756263,0
2,20160328170001757443,1
3,20160328190001761939,1
4,20160329140001784605,1
...,...,...
120924,616776554336227592,1
120925,616776979319886088,0
120926,616778213804216328,0
120927,616780774938513416,0


In [62]:
# count number of phones each id has
tmp_receive_phone_count = train_receive_info.groupby(by=['id']).apply(lambda x: x['fix_phone'].count()
                                                                     ).reset_index(name='receive_phone_count')
tmp_receive_phone_count

,id,receive_phone_count
0,20160328090001743085,0
1,20160328160001756263,0
2,20160328170001757443,2
3,20160328190001761939,1
4,20160329140001784605,5
...,...,...
120924,616776554336227592,1
120925,616776979319886088,0
120926,616778213804216328,0
120927,616780774938513416,0


In [113]:
# create new feature to measure whether user like to fill his phone number
fxpho_num = train_receieve_info.groupby(by='id')['fix_phone'].nunique()
fxpho_rate = train_recieve_info.groupby('id').apply(lambda x: x.fix_phone.count()/(x.phone.count() + 1e-6))
#train_recieve_info['fxpho_num'] = train_recieve_info.id.map(fxpho_num)
#train_recieve_info['fxpho_rate'] = train_recieve_info.id.map(fxpho_rate)

In [79]:
train_receive_info.head()

,addr_id,region,phone,fix_phone,id,receiver_md5
0,NaN,unknown,NaN,None,5.751500e+17,NaN
1,137816020.0,LN,186****4600,186****4600,5.638880e+17,798371eaefb014d304e8c90ba0af1d09
2,137630418.0,LN,159****1000,159****1000,5.638880e+17,41c14ef94733b4587f11ffb64e58a72d
3,137816021.0,LN,186****4600,None,5.638880e+17,798371eaefb014d304e8c90ba0af1d09
4,138135925.0,LN,189****9687,189****9687,5.638880e+17,2d3e7dc595265a37f1e9eee82a15b047


### train_target

In [7]:
train_target = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/train_target.csv",
                           parse_dates=['appl_sbm_tm'])

In [34]:
train_target.head()

,appl_sbm_tm,target,id
0,2017-04-08 19:10:13,0,600689284231467016
1,2016-06-08 15:20:21,0,20160608150003808365
2,2016-04-27 11:36:27,0,20160427110002485018
3,2017-03-30 08:45:55,0,593852820159991816
4,2016-11-30 11:40:38,0,506970882778861832


In [117]:
# train_target['target']=train_target['target'].astype('object')

In [118]:
train_target.describe()

,appl_sbm_tm,target,id
count,120929,120929,120929
unique,119798,2,120929
top,2017-03-04 15:30:18.0,0,576511157116866568
freq,3,117541,1


### user_info

In [34]:
train_user_info=pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/user_info.csv")
#train_user_info = train_user_info.drop(columns = "Unnamed: 0")

In [35]:
train_user_info.head()

,sex,birthday,merriage,income,id_card,degree,qq_bound,wechat_bound,account_grade,id
0,No Disclosure,NaN,unknown,unknown,NaN,unknown,Not Bounded,Bounded,3rd Level,588238879438409736
1,No Disclosure,NaN,unknown,unknown,NaN,unknown,Bounded,Not Bounded,3rd Level,601460242772922632
2,No Disclosure,NaN,unknown,unknown,NaN,unknown,Not Bounded,Bounded,1st Level,601137941405765640
3,No Disclosure,NaN,unknown,unknown,NaN,unknown,Bounded,Not Bounded,Initial Level,20160429090002533592
4,Male,1994-10-04,unknown,unknown,NaN,unknown,Bounded,Not Bounded,Initial Level,613466575764197640


In [19]:
train_user_info.dtypes

sex              object
birthday         object
merriage         object
income           object
id_card          object
degree           object
qq_bound         object
wechat_bound     object
account_grade    object
id               object
dtype: object

In [36]:
# create new feature to measure whether user miss id_card; 0: missing, 1: not missing
is_idcard = train_user_info['id_card'].map(lambda x: 0 if str(x) == 'nan' else 1)
is_idcard_df = pd.DataFrame()
is_idcard_df['id'] = train_user_info['id']
is_idcard_df['is_idcard'] = is_idcard
is_idcard_df.head()

,id,is_idcard
0,588238879438409736,0
1,601460242772922632,0
2,601137941405765640,0
3,20160429090002533592,0
4,613466575764197640,0


In [37]:
# process user birthday
tmp = train_user_info[['id', 'birthday']]
tmp = tmp.set_index(['id'])
tmp.head()

,birthday
id,
588238879438409736,NaN
601460242772922632,NaN
601137941405765640,NaN
20160429090002533592,NaN
613466575764197640,1994-10-04


In [38]:
tmp[tmp['birthday']=='1-1-1']

,birthday
id,


In [39]:
is_double = tmp['birthday'].map(lambda x: (str(x)=='--')*1).reset_index(name='is_double')
is_double.head()

,id,is_double
0,588238879438409736,0
1,601460242772922632,0
2,601137941405765640,0
3,20160429090002533592,0
4,613466575764197640,0


In [40]:
is_0_0_0 = tmp['birthday'].map(lambda x: (str(x)=='0-0-0')*1).reset_index(name='is_0_0_0')
is_0_0_0.head()

,id,is_0_0_0
0,588238879438409736,0
1,601460242772922632,0
2,601137941405765640,0
3,20160429090002533592,0
4,613466575764197640,0


In [41]:
is_0000_00_00 = tmp['birthday'].map(lambda x: (str(x)=='0000-00-00')*1).reset_index(name='is_0000_00_00')
is_0001_1_1 = tmp['birthday'].map(lambda x: (str(x)=='0001-1-1')*1).reset_index(name='is_0001_1_1')
is_nan = tmp['birthday'].map(lambda x:(str(x)=='nan')*1).reset_index(name='is_nan')
is_nan.head()

,id,is_nan
0,588238879438409736,1
1,601460242772922632,1
2,601137941405765640,1
3,20160429090002533592,1
4,613466575764197640,0


In [44]:
train_user_info['birthday'].unique()

array([                          'NaT', '1994-10-04T00:00:00.000000000',
       '1992-10-24T00:00:00.000000000', ...,
       '1965-06-15T00:00:00.000000000', '1973-01-24T00:00:00.000000000',
       '1975-05-31T00:00:00.000000000'], dtype='datetime64[ns]')

In [43]:
# convert birthday to datetime.type
train_user_info['birthday'] = train_user_info['birthday'].map(
    lambda x: datetime.datetime.strptime(str(x), '%Y-%m-%d')
    if (re.match('19\d{2}-\d{1,2}-\d{1,2}', str(x))) else pd.NaT)
train_user_info['birthday'].head()

# re.match('19\d{2}-\d{1,2}-\d{1,2}', str(x)) and '-0' not in str(x)

0          NaT
1          NaT
2          NaT
3          NaT
4   1994-10-04
Name: birthday, dtype: datetime64[ns]

In [28]:
train_user_info.describe()

,id
count,1.209290e+05
mean,3.353065e+18
std,6.836821e+18
min,4.402520e+17
25%,5.732000e+17
50%,5.869260e+17
75%,6.084450e+17
max,2.016080e+19


In [27]:
train_user_info.to_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/user_info.csv", index=False)

In [32]:
train_user_info=pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/user_info.csv")
train_user_info.dtypes

sex              object
birthday         object
merriage         object
income           object
id_card          object
degree           object
qq_bound         object
wechat_bound     object
account_grade    object
id               object
dtype: object

In [29]:
train_user_info.head()

,sex,birthday,merriage,income,id_card,degree,qq_bound,wechat_bound,account_grade,id
0,No Disclosure,NaT,unknown,unknown,NaN,unknown,Not Bounded,Bounded,3rd Level,588238879438409736
1,No Disclosure,NaT,unknown,unknown,NaN,unknown,Bounded,Not Bounded,3rd Level,601460242772922632
2,No Disclosure,NaT,unknown,unknown,NaN,unknown,Not Bounded,Bounded,1st Level,601137941405765640
3,No Disclosure,NaT,unknown,unknown,NaN,unknown,Bounded,Not Bounded,Initial Level,20160429090002533592
4,Male,1994-10-04,unknown,unknown,NaN,unknown,Bounded,Not Bounded,Initial Level,613466575764197640


### Merge Features

In [12]:
auth_info = auth_info.drop(columns = "Unnamed: 0")

In [14]:
auth_info.to_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/auth_info.csv", index=False)

In [2]:
auth_info=pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/auth_info.csv",
                      parse_dates=['auth_time'])

In [3]:
credit_info=pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/credit_info.csv")

In [66]:
bankcard_info = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/bankcard_info.csv")

In [67]:
bankcard_info.head()

,bank_name,tail_num,card_type,phone,id,card_type_count,bankcard_count,phone_count
0,citic,9753,debit,China Telecom,585401206734589960,2,3,2
1,abc,2174,debit,China Mobile,587694980785508360,2,4,1
2,bcom,4832,credit,China Unicom,602098357531316232,2,16,2
3,bob,5697,debit,China Unicom,567205103669350408,2,3,1
4,citic,7703,debit,China Mobile,577538085852549384,2,6,1


In [25]:
train_data = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/train_data.csv",
                        parse_dates=['appl_sbm_tm'])

In [3]:
order_info=pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/order_info.csv")

In [4]:
order_info.shape

(2415645, 21)

In [6]:
train_user_info=pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/user_info.csv")

In [26]:
hobby_ind = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/user_hobby_industry_info.csv")

In [12]:
train_target = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/train_target.csv",
                           parse_dates=['appl_sbm_tm'])

In [10]:
tmp = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/AI_risk_train_V3.0/train_target.csv")

In [48]:
train_user_info.head()

,sex,birthday,merriage,income,id_card,degree,qq_bound,wechat_bound,account_grade,id
0,No Disclosure,NaT,unknown,unknown,NaN,unknown,Not Bounded,Bounded,3rd Level,588238879438409736
1,No Disclosure,NaT,unknown,unknown,NaN,unknown,Bounded,Not Bounded,3rd Level,601460242772922632
2,No Disclosure,NaT,unknown,unknown,NaN,unknown,Not Bounded,Bounded,1st Level,601137941405765640
3,No Disclosure,NaT,unknown,unknown,NaN,unknown,Bounded,Not Bounded,Initial Level,20160429090002533592
4,Male,1994-10-04,unknown,unknown,NaN,unknown,Bounded,Not Bounded,Initial Level,613466575764197640


In [30]:
train_data = pd.merge(train_target ,auth_info, on=['id'], how='left')
train_data = pd.merge(train_data ,train_user_info, on=['id'], how='left')
train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card,phone_exist,auth_time_year,auth_time_month,auth_time_day,sex,birthday,merriage,income,id_card_y,degree,qq_bound,wechat_bound,account_grade
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,5,Male,1992-06-15,unknown,unknown,NaN,unknown,Bounded,Bounded,unknown
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,31,No Disclosure,NaT,unknown,unknown,NaN,unknown,Not Bounded,Bounded,unknown
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,18,Male,1978-06-25,unknown,unknown,NaN,unknown,Bounded,Bounded,3rd Level
3,2017-03-30 08:45:55,0,593852820159991816,NaN,NaT,China Unicom,0,1,-1,-1,-1,No Disclosure,1987-01-09,unknown,unknown,NaN,unknown,Bounded,Bounded,unknown
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,15,Male,1985-12-08,unknown,unknown,NaN,unknown,Not Bounded,Bounded,unknown


In [31]:
train_data = pd.merge(train_data ,credit_info, on=['id'], how='left')
train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card,phone_exist,auth_time_year,auth_time_month,...,degree,qq_bound,wechat_bound,account_grade,credit_score,overdraft,quota,remaining,using_rate,inverse_credit_score
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,...,unknown,Bounded,Bounded,unknown,394.0,0.00,0.0,0.00,0.000000,211.0
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,...,unknown,Not Bounded,Bounded,unknown,578.0,2647.14,4416.0,1768.86,0.599443,27.0
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,...,unknown,Bounded,Bounded,3rd Level,402.0,0.00,0.0,0.00,0.000000,203.0
3,2017-03-30 08:45:55,0,593852820159991816,NaN,NaT,China Unicom,0,1,-1,-1,...,unknown,Bounded,Bounded,unknown,579.0,15125.00,15146.0,21.00,0.998613,26.0
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,...,unknown,Not Bounded,Bounded,unknown,390.0,0.00,0.0,0.00,0.000000,215.0


In [32]:
train_data['hour'] = train_data['appl_sbm_tm'].map(lambda x: x.hour)
train_data['month'] = train_data['appl_sbm_tm'].map(lambda x: x.month)
train_data['year'] = train_data['appl_sbm_tm'].map(lambda x: x.year)
train_data['nan_num'] = train_data.isnull().sum(axis=1)

In [33]:
# Have negative values
train_data['diff_day'] = train_data.apply(lambda row: (row['appl_sbm_tm']-row['auth_time']).days, axis=1)
train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card,phone_exist,auth_time_year,auth_time_month,...,overdraft,quota,remaining,using_rate,inverse_credit_score,hour,month,year,nan_num,diff_day
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,...,0.00,0.0,0.00,0.000000,211.0,19,4,2017,1,34.0
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,...,2647.14,4416.0,1768.86,0.599443,27.0,15,6,2016,2,129.0
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,...,0.00,0.0,0.00,0.000000,203.0,11,4,2016,1,-417.0
3,2017-03-30 08:45:55,0,593852820159991816,NaN,NaT,China Unicom,0,1,-1,-1,...,15125.00,15146.0,21.00,0.998613,26.0,8,3,2017,3,NaN
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,...,0.00,0.0,0.00,0.000000,215.0,11,11,2016,1,-380.0


In [34]:
train_data['how_old'] = train_data.apply(lambda row: (row['appl_sbm_tm']-row['birthday']).days/365, axis=1)
train_data.head()

0    24.830137
1          NaN
2    37.865753
3    30.241096
4    31.000000
Name: how_old, dtype: float64

In [35]:
train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card,phone_exist,auth_time_year,auth_time_month,...,quota,remaining,using_rate,inverse_credit_score,hour,month,year,nan_num,diff_day,how_old
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,...,0.0,0.00,0.000000,211.0,19,4,2017,1,34.0,24.830137
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,...,4416.0,1768.86,0.599443,27.0,15,6,2016,2,129.0,NaN
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,...,0.0,0.00,0.000000,203.0,11,4,2016,1,-417.0,37.865753
3,2017-03-30 08:45:55,0,593852820159991816,NaN,NaT,China Unicom,0,1,-1,-1,...,15146.0,21.00,0.998613,26.0,8,3,2017,3,NaN,30.241096
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,...,0.0,0.00,0.000000,215.0,11,11,2016,1,-380.0,31.000000


In [36]:
train_data.to_csv(
    "C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/train_data.csv", index=False)

In [79]:
train_data[['birthday', 'appl_sbm_tm', 'id_card_y']].head()

,birthday,appl_sbm_tm,id_card_y
0,1992-06-15,2017-04-08 19:10:13,NaN
1,NaT,2016-06-08 15:20:21,NaN
2,1978-06-25,2016-04-27 11:36:27,NaN
3,1987-01-09,2017-03-30 08:45:55,NaN
4,1985-12-08,2016-11-30 11:40:38,NaN


In [37]:
train_data['id_card_y'].unique()

array([nan, '320******032', '412******032', ..., '460******050',
       '440******118', '330******751'], dtype=object)

In [38]:
train_data['id_card_x'].unique()

array(['4****************2', '2****************3', '4****************7',
       ..., '34**************56', '36073********658', '44122********057'],
      dtype=object)

In [39]:
# check whether user has the same id_card 
auth_idcard = list(train_data['id_card_x'])
user_idcard = list(train_data['id_card_y'])
idcard_result = []
for indexx, uu in enumerate(auth_idcard):
    if(str(auth_idcard[indexx]) == 'nan') and (str(user_idcard[indexx]) == 'nan'):
        idcard_result.append(0)
    elif (str(auth_idcard[indexx]) != 'nan') and (str(user_idcard[indexx]) == 'nan'):
        idcard_result.append(1)
    elif (str(auth_idcard[indexx]) == 'nan') and (str(user_idcard[indexx]) != 'nan'):
        idcard_result.append(2)
    else:
        ttt1 = str(auth_idcard[indexx])[0] + str(auth_idcard[indexx])[-1]
        ttt2 = str(user_idcard[indexx])[0] + str(user_idcard[indexx])[-1]
        if ttt1 == ttt2:
            idcard_result.append(3)
        if ttt1 != ttt2:
            idcard_result.append(4)
train_data['the_same_id'] = idcard_result
train_data['the_same_id'].head()

0    1
1    1
2    1
3    0
4    1
Name: the_same_id, dtype: int64

In [41]:
train_data['the_same_id'].unique()

array([1, 0, 3, 2, 4], dtype=int64)

In [68]:
bankcard_info.head()

,bank_name,tail_num,card_type,phone,id,card_type_count,bankcard_count,phone_count
0,citic,9753,debit,China Telecom,585401206734589960,2,3,2
1,abc,2174,debit,China Mobile,587694980785508360,2,4,1
2,bcom,4832,credit,China Unicom,602098357531316232,2,16,2
3,bob,5697,debit,China Unicom,567205103669350408,2,3,1
4,citic,7703,debit,China Mobile,577538085852549384,2,6,1


In [69]:
bankcard_phone_list = bankcard_info.groupby(by=['id'])['phone'].apply(
    lambda x: list(set(x.tolist()))).reset_index(name='bank_phone_list')
bankcard_phone_list.head()

,id,bank_phone_list
0,20160328090001743085,[China Mobile]
1,20160328160001756263,[China Mobile]
2,20160328170001757443,[China Mobile]
3,20160328190001761939,[China Mobile]
4,20160329140001784605,"[China Unicom, China Mobile]"


In [46]:
# merge features
train_data = pd.merge(train_data, bankcard_phone_list, on=['id'], how='left')
train_data['exist_phone'] = train_data.apply(lambda x: x['phone'] in x['bank_phone_list'], axis=1)
train_data['exist_phone'] = train_data['exist_phone']*1

In [47]:
train_data = train_data.drop(['bank_phone_list'], axis=1)

In [48]:
train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card,phone_exist,auth_time_year,auth_time_month,...,using_rate,inverse_credit_score,hour,month,year,nan_num,diff_day,how_old,the_same_id,exist_phone
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,...,0.000000,211.0,19,4,2017,1,34.0,24.830137,1,1
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,...,0.599443,27.0,15,6,2016,2,129.0,NaN,1,1
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,...,0.000000,203.0,11,4,2016,1,-417.0,37.865753,1,1
3,2017-03-30 08:45:55,0,593852820159991816,NaN,NaT,China Unicom,0,1,-1,-1,...,0.998613,26.0,8,3,2017,3,NaN,30.241096,0,1
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,...,0.000000,215.0,11,11,2016,1,-380.0,31.000000,1,1


In [50]:
"""bank_info"""
bank_name = bankcard_info.groupby(by=['id'], as_index=False)['bank_name'].agg({'bank_name_len': lambda x: len(set(x))})
bank_name.head()

,id,bank_name_len
0,20160328090001743085,3
1,20160328160001756263,5
2,20160328170001757443,1
3,20160328190001761939,2
4,20160329140001784605,7


In [71]:
bank_num = bankcard_info.groupby(by=['id'], as_index=False)['tail_num'].agg({'tail_num_len': lambda x: len(set(x))})
bank_num.head()

,id,tail_num_len
0,20160328090001743085,4
1,20160328160001756263,7
2,20160328170001757443,1
3,20160328190001761939,2
4,20160329140001784605,10


In [70]:
bank_phone_num = bankcard_info.groupby(by=['id'], as_index=False)['phone'].agg({'bank_phone_num': lambda x:x.nunique()})
bank_phone_num.head()

,id,bank_phone_num
0,20160328090001743085,1
1,20160328160001756263,1
2,20160328170001757443,1
3,20160328190001761939,1
4,20160329140001784605,2


In [22]:
train_data['exist_phone'] = train_data['exist_phone']*1
train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card,phone_exist,auth_time_year,auth_time_month,...,using_rate,inverse_credit_score,hour,month,year,nan_num,diff_day,how_old,the_same_id,exist_phone
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05 00:00:00,China Mobile,1,1,2017,3,...,0.000000,211.0,19,4,2017,1,34.0,24.830137,1,1
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31 00:00:00,China Mobile,1,1,2016,1,...,0.599443,27.0,15,6,2016,2,129.0,NaN,1,1
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18 00:00:00,China Mobile,1,1,2017,6,...,0.000000,203.0,11,4,2016,1,-417.0,37.865753,1,1
3,2017-03-30 08:45:55,0,593852820159991816,NaN,NaN,China Unicom,0,1,-1,-1,...,0.998613,26.0,8,3,2017,3,NaN,30.241096,0,1
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15 00:00:00,China Mobile,1,1,2017,12,...,0.000000,215.0,11,11,2016,1,-380.0,31.000000,1,1


In [55]:
order_info = order_info.drop(columns = "Unnamed: 0")
order_info.head()

,amt_order,type_pay,time_order,sts_order,phone,unit_price,id,no_order_md5,name_rec_md5,product_id_md5,order_sum,order_mean,order_std,order_skew,pho_num,rec_num,Cash on Delivery,Online Pay,order_time_min,order_time_max
0,39.0,Online Pay,2017-04-15 09:21:18,Waiting for Receipt,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
1,39.0,Online Pay,2017-04-15 09:21:18,Finished,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,NaN,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
2,288.0,Online Pay,2016-10-24 08:57:06,unknown,137****8223,NaN,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,NaN,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
3,428.0,Online Pay,2016-10-27 20:04:48,Finished,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
4,428.0,Online Pay,2016-10-27 20:04:48,unknown,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,NaN,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18


In [20]:
#train_data = pd.merge(train_data, order_info[['id', 'order_time_max']], on=['id'], how='left')
#train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card,phone_exist,auth_time_year,auth_time_month,...,inverse_credit_score,hour,month,year,nan_num,diff_day,how_old,the_same_id,exist_phone,order_time_max
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05 00:00:00,China Mobile,1,1,2017,3,...,211.0,19,4,2017,1,34.0,24.830137,1,1,2016-01-25 15:39:12
1,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05 00:00:00,China Mobile,1,1,2017,3,...,211.0,19,4,2017,1,34.0,24.830137,1,1,2016-01-25 15:39:12
2,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05 00:00:00,China Mobile,1,1,2017,3,...,211.0,19,4,2017,1,34.0,24.830137,1,1,2016-01-25 15:39:12
3,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05 00:00:00,China Mobile,1,1,2017,3,...,211.0,19,4,2017,1,34.0,24.830137,1,1,2016-01-25 15:39:12
4,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31 00:00:00,China Mobile,1,1,2016,1,...,27.0,15,6,2016,2,129.0,NaN,1,1,2016-05-24 15:37:46


In [83]:
order_info[['id', 'order_time_min']].head()

,id,order_time_min
0,609460154345197832,2016-02-20 12:09:50
1,609460154345197832,2016-02-20 12:09:50
2,609460154345197832,2016-02-20 12:09:50
3,609460154345197832,2016-02-20 12:09:50
4,609460154345197832,2016-02-20 12:09:50


In [73]:
hobby_ind = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/user_hobby_industry_info.csv")

In [74]:
hobby_ind.head()

,id,is_hobby,is_industry
0,588238879438409736,0,0
1,601460242772922632,0,0
2,601137941405765640,0,0
3,20160429090002533592,0,0
4,613466575764197640,0,0


In [75]:
hobby_ind.shape

(120929, 3)

In [88]:
train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card_x,phone_exist_x,auth_time_year,auth_time_month,...,XZ,YN,ZJ,unknown,receive_phone_count,unique_fix_phone,bank_phone_num,is_hobby,is_industry,is_idcard
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,...,0,0,0,0,0,0,1,0,0,0
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,...,0,0,0,0,3,3,1,0,0,0
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,...,0,0,0,0,1,1,1,1,0,0
3,2017-03-30 08:45:55,0,593852820159991816,NaN,NaN,China Unicom,0,1,-1,-1,...,0,0,0,0,1,1,1,0,0,0
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,...,0,0,0,0,2,1,1,0,0,0


In [85]:
train_data = pd.merge(train_data, auth_info[['id', 'spt_id_card']], on=['id'], how='left')
train_data = pd.merge(train_data, auth_info[['id', 'phone_exist']], on=['id'], how='left')
train_data.drop()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card_x,phone_exist_x,auth_time_year,auth_time_month,...,ZJ,unknown,receive_phone_count,unique_fix_phone,bank_phone_num,is_hobby,is_industry,is_idcard,spt_id_card_y,phone_exist_y
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,...,0,0,0,0,1,0,0,0,1,1
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,...,0,0,3,3,1,0,0,0,1,1
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,...,0,0,1,1,1,1,0,0,1,1
3,2017-03-30 08:45:55,0,593852820159991816,NaN,NaN,China Unicom,0,1,-1,-1,...,0,0,1,1,1,0,0,0,0,1
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,...,0,0,2,1,1,0,0,0,1,1


In [87]:
train_data = train_data.drop(columns='spt_id_card_y')
train_data = train_data.drop(columns='phone_exist_y')

In [3]:
train_data = pd.read_csv("C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/train_data.csv",
                        parse_dates=['appl_sbm_tm'])

In [128]:
train_data.to_csv(
    "C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/train_data.csv", index=False)

In [110]:
train_data['order_time_min'].head()

0    2016-01-18 23:55:44
1    2012-10-18 12:40:43
2    2014-06-11 18:07:34
3    2016-01-24 23:13:59
4                     -1
Name: order_time_min, dtype: object

In [101]:
train_data['order_time_max'] = train_data['order_time_max'].map(
    lambda x: pd.NaT if str(x) == '-1' else(pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S')))

In [103]:
# merge
#train_data = pd.merge(train_data, bank_name, on=['id'], how='left')
#train_data = pd.merge(train_data, bank_num, on=['id'], how='left')
#train_data = pd.merge(train_data, train_order_info_time_max, on=['id'], how='left')
#train_data = pd.merge(train_data, train_order_time_min, on=['id'], how='left')
#train_data = pd.merge(train_data, train_order_online_pay, on=['id'], how='left')
#train_data = pd.merge(train_data, train_order_cash_on_deliv, on=['id'], how='left')
#train_data = pd.merge(train_data, is_double, on=['id'], how='left')
#train_data = pd.merge(train_data, is_0_0_0, on=['id'], how='left')
#train_data = pd.merge(train_data, is_0000_00_00, on=['id'], how='left')
#train_data = pd.merge(train_data, is_0001_1_1, on=['id'], how='left')
#train_data = pd.merge(train_data, tmp_receive, on=['id'], how='left')
#train_data = pd.merge(train_data, tmp_receive_phone_count, on=['id'], how='left')
#train_data = pd.merge(train_data, fxpho_num, on=['id'], how='left')
#train_data = pd.merge(train_data, bank_phone_num, on=['id'], how='left')
#train_data = pd.merge(train_data, hobby_ind[['id', 'is_hobby']], on=['id'], how='left')
#train_data = pd.merge(train_data, is_idcard_df, on=['id'], how='left')
train_data['day_order_max'] = train_data.apply(lambda row: (row['appl_sbm_tm'] - row['order_time_max']).days, axis=1)

In [106]:
train_data = train_data.drop(['order_time_max'], axis=1)

In [111]:
train_data['order_time_min'] = train_data['order_time_min'].map(
    lambda x: pd.NaT if str(x) == '-1' else(pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S')))
train_data['day_order_min'] = train_data.apply(lambda row: (row['appl_sbm_tm'] - row['order_time_min']).days, axis=1)

In [113]:
train_data = train_data.drop(['order_time_min'], axis=1)

In [115]:
train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card_x,phone_exist_x,auth_time_year,auth_time_month,...,ZJ,unknown,receive_phone_count,unique_fix_phone,bank_phone_num,is_hobby,is_industry,is_idcard,day_order_max,day_order_min
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,...,0,0,0,0,1,0,0,0,439.0,445.0
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,...,0,0,3,3,1,0,0,0,14.0,1329.0
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,...,0,0,1,1,1,1,0,0,685.0,685.0
3,2017-03-30 08:45:55,0,593852820159991816,NaN,NaN,China Unicom,0,1,-1,-1,...,0,0,1,1,1,0,0,0,2.0,430.0
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,...,0,0,2,1,1,0,0,0,NaN,NaN


In [118]:
order_info.head()

,Unnamed: 0,amt_order,type_pay,time_order,sts_order,phone,unit_price,id,no_order_md5,name_rec_md5,...,order_sum,order_mean,order_std,order_skew,pho_num,rec_num,Cash on Delivery,Online Pay,order_time_min,order_time_max
0,0,39.0,Online Pay,2017-04-15 09:21:18,Waiting for Receipt,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,...,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
1,1,39.0,Online Pay,2017-04-15 09:21:18,Finished,159****2937,NaN,609460154345197832,ae3b7fd144ff7ee8658d92137a924836,afeb593025b37afbe8179c456b8855f1,...,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
2,2,288.0,Online Pay,2016-10-24 08:57:06,unknown,137****8223,NaN,609460154345197832,1d7b1294cdbfe24815e39c458cd059d1,eac9416f834d481d7c88da6134468d03,...,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
3,3,428.0,Online Pay,2016-10-27 20:04:48,Finished,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,...,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18
4,4,428.0,Online Pay,2016-10-27 20:04:48,unknown,137****8223,NaN,609460154345197832,51ee194476e14e9976db834c08f20e21,eac9416f834d481d7c88da6134468d03,...,1828.0,228.5,158.817955,-0.011212,2,2,3,5,2016-02-20 12:09:50,2017-04-15 09:21:18


In [120]:
# order info
order_time = order_info.groupby(['id'], as_index=False)['amt_order'].agg({'order_time': len})
order_time.head()

,id,order_time
0,20160328090001743085,6.0
1,20160328160001756263,1.0
2,20160328170001757443,1.0
3,20160328190001761939,41.0
4,20160329140001784605,33.0


In [121]:
unit_price_mean = order_info.groupby(['id'], as_index=False)['unit_price'].agg({'unit_price_mean': np.mean})
unit_price_mean.head()

,id,unit_price_mean
0,20160328090001743085,1591.166667
1,20160328160001756263,NaN
2,20160328170001757443,NaN
3,20160328190001761939,459.256098
4,20160329140001784605,352.493103


In [122]:
order_time_set = order_info.groupby(['id'], as_index=False)['time_order'].agg({'order_time_set': lambda x: len(set(x))})
order_time_set.head()

,id,order_time_set
0,20160328090001743085,2
1,20160328160001756263,1
2,20160328170001757443,1
3,20160328190001761939,29
4,20160329140001784605,19


In [124]:
train_data = pd.merge(train_data, order_time, on=['id'], how='left')
train_data = pd.merge(train_data, order_mean, on=['id'], how='left')
train_data = pd.merge(train_data, unit_price_mean, on=['id'], how='left')
train_data = pd.merge(train_data, order_time_set, on=['id'], how='left')

In [126]:
train_data = pd.merge(train_data, order_std, on=['id'], how='left')
train_data = pd.merge(train_data, order_skew, on=['id'], how='left')
train_data = pd.merge(train_data, order_sum, on=['id'], how='left')

In [4]:
train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card_x,phone_exist_x,auth_time_year,auth_time_month,...,is_idcard,day_order_max,day_order_min,order_time,order_mean,unit_price_mean,order_time_set,order_std,order_skew,order_sum
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,...,0,439.0,445.0,4.0,95.750000,NaN,3,70.196747,1.653270,383.00
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,...,0,14.0,1329.0,13.0,387.042308,254.970833,11,576.079251,1.719580,5031.55
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,...,0,685.0,685.0,2.0,1837.000000,918.500000,1,0.000000,NaN,3674.00
3,2017-03-30 08:45:55,0,593852820159991816,NaN,NaN,China Unicom,0,1,-1,-1,...,0,2.0,430.0,60.0,1125.125833,NaN,56,1855.334525,1.580385,67507.55
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,...,0,NaN,NaN,1.0,NaN,NaN,1,NaN,NaN,0.00


In [127]:
train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card_x,phone_exist_x,auth_time_year,auth_time_month,...,is_idcard,day_order_max,day_order_min,order_time,order_mean,unit_price_mean,order_time_set,order_std,order_skew,order_sum
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,...,0,439.0,445.0,4.0,95.750000,NaN,3,70.196747,1.653270,383.00
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,...,0,14.0,1329.0,13.0,387.042308,254.970833,11,576.079251,1.719580,5031.55
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,...,0,685.0,685.0,2.0,1837.000000,918.500000,1,0.000000,NaN,3674.00
3,2017-03-30 08:45:55,0,593852820159991816,NaN,NaN,China Unicom,0,1,-1,-1,...,0,2.0,430.0,60.0,1125.125833,NaN,56,1855.334525,1.580385,67507.55
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,...,0,NaN,NaN,1.0,NaN,NaN,1,NaN,NaN,0.00


In [5]:
dummy_fea = ['sex', 'merriage', 'income', 'qq_bound', 'degree', 'wechat_bound', 'account_grade']
dummy_df = pd.get_dummies(train_data.loc[:,dummy_fea])
# add one-hot features
train_data_copy = pd.concat([train_data, dummy_df], axis=1)
train_data_copy = train_data_copy.fillna(0)
train_data_copy

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card_x,phone_exist_x,auth_time_year,auth_time_month,...,degree_unknown,wechat_bound_Bounded,wechat_bound_Not Bounded,wechat_bound_unknown,account_grade_1st Level,account_grade_2nd Level,account_grade_3rd Level,account_grade_Initial Level,account_grade_Premier Level,account_grade_unknown
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,...,1,1,0,0,0,0,0,0,0,1
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,...,1,1,0,0,0,0,0,0,0,1
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,...,1,1,0,0,0,0,1,0,0,0
3,2017-03-30 08:45:55,0,593852820159991816,0,0,China Unicom,0,1,-1,-1,...,1,1,0,0,0,0,0,0,0,1
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,...,1,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120924,2017-03-26 00:45:23,0,590710366225305608,0,0,China Mobile,0,1,-1,-1,...,1,0,1,0,0,0,0,0,0,1
120925,2017-03-09 16:00:07,0,578853574977523976,0,0,China Mobile,0,1,-1,-1,...,1,0,1,0,0,0,0,0,0,1
120926,2017-04-24 15:47:39,0,612185913824382984,3****************2,2017-08-14,China Mobile,1,1,2017,8,...,0,0,1,0,0,0,0,1,0,0
120927,2016-11-10 22:17:21,0,492795832240705800,4****************6,0,0,1,0,-1,-1,...,1,0,1,0,0,0,1,0,0,0


In [6]:
# delete features that are used for one-hot encoding
valid_train_data = train_data_copy.drop(dummy_fea, axis=1)

In [7]:
valid_train_data.head()

,appl_sbm_tm,target,id,id_card_x,auth_time,phone,spt_id_card_x,phone_exist_x,auth_time_year,auth_time_month,...,degree_unknown,wechat_bound_Bounded,wechat_bound_Not Bounded,wechat_bound_unknown,account_grade_1st Level,account_grade_2nd Level,account_grade_3rd Level,account_grade_Initial Level,account_grade_Premier Level,account_grade_unknown
0,2017-04-08 19:10:13,0,600689284231467016,4****************2,2017-03-05,China Mobile,1,1,2017,3,...,1,1,0,0,0,0,0,0,0,1
1,2016-06-08 15:20:21,0,20160608150003808365,2****************3,2016-01-31,China Mobile,1,1,2016,1,...,1,1,0,0,0,0,0,0,0,1
2,2016-04-27 11:36:27,0,20160427110002485018,4****************7,2017-06-18,China Mobile,1,1,2017,6,...,1,1,0,0,0,0,1,0,0,0
3,2017-03-30 08:45:55,0,593852820159991816,0,0,China Unicom,0,1,-1,-1,...,1,1,0,0,0,0,0,0,0,1
4,2016-11-30 11:40:38,0,506970882778861832,6****************2,2017-12-15,China Mobile,1,1,2017,12,...,1,1,0,0,0,0,0,0,0,1


In [199]:
valid_train_data.to_csv(
    "C:/Users/Vance He/Desktop/UMich/2020_WN/STATS507/project/project/data/processed_data/train_data_one_hot.csv", index=False)

In [176]:
# split train set and test set
train_set = valid_train_data[valid_train_data['appl_sbm_tm'] < datetime.datetime(2017,3,30)]
test_set = valid_train_data[valid_train_data['appl_sbm_tm'] >= datetime.datetime(2017,3,30)]

In [197]:
valid_train_data.auth_time_day = valid_train_data.auth_time_day.apply(lambda x: 0 if x== -1 else x)

In [198]:
valid_train_data.auth_time_day.head()

0     5
1    31
2    18
3     0
4    15
Name: auth_time_day, dtype: int64

In [189]:
a.head()

0    2017
1    2016
2    2017
3       0
4    2017
Name: auth_time_year, dtype: int64

In [170]:
# latest time: 2017-04-30
# earliest time: 2016-03-28
max(valid_train_data.appl_sbm_tm)

Timestamp('2017-04-30 23:59:54')

In [171]:
valid_train_data.shape

(120929, 119)

In [177]:
test_set.shape

(33755, 119)

In [182]:
sum(test_set['target'][test_set['target']==1])

975